## Loading credentials from another config file

In [1]:
import config
import pandas as pd
import requests
import spotipy
import pickle
from sklearn.preprocessing import StandardScaler
import json
from IPython.display import IFrame

## Initializing Spotify API

In [2]:
from spotipy.oauth2 import SpotifyClientCredentials


#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

## Loading the clustered data frame and the respective model

In [3]:
#Loading the clustered data frame and the respective model

clustered_df = pd.read_csv('test_PCA.csv')

with open("test_PCA.pkl", "rb") as f:
    scaler, kmeans, pca = pickle.load(f)


In [4]:
len(clustered_df['cluster'].unique())

7

In [5]:
kmeans.inertia_

58301.20302801356

## defining functions

In [6]:
def play(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [7]:
def search():
    # Get user input for song name
    song_name = input("Enter the name of a song: ")

    # Use the Spotify search API to find songs matching the user input
    results = sp.search(q=song_name, type='track')

    # Display the top matching songs to the user
    for i, track in enumerate(results['tracks']['items'][:10]):
        print(f"{i+1}. {track['name']} by {track['artists'][0]['name']}")

    # Ask the user to select the song they meant from the list
    selected_song = None
    while not selected_song:
        selection = input("Please select the song you meant by number (or type 'exit' to cancel): ")
        if selection == 'exit':
            return
        try:
            selected_song = results['tracks']['items'][int(selection)-1]
        except:
            print("Invalid selection, please try again.")

    # Get the track ID of the selected song
    track_id = selected_song['id']

    # Display the selected song using an embedded player
    print(f"Now playing: {selected_song['name']} by {selected_song['artists'][0]['name']}")
    display(play(track_id))

    return(track_id)

In [8]:
def recommend(track_id):
    # Use the Spotify API to get the audio features for the selected song
    audio_features = sp.audio_features(track_id)[0]

    # Transform the audio features using the same PCA used to transform the clustered_df
    X = pd.DataFrame(audio_features, index=[0])
    X = X.drop(columns=['type','uri','track_href','analysis_url','id'])


    X_pca = pca.transform(scaler.transform(X))

    # Use the KMeans model to predict the cluster for the selected song
    cluster = kmeans.predict(X_pca)[0]

    # Filter for songs in the same cluster as the selected song
    recommended_songs = clustered_df[clustered_df['cluster'] == cluster]

    # Recommend a random song from the list of songs in the same cluster as the selected song
    recommended_song_id = recommended_songs.sample()['id'].values[0]

    # Get the recommended song name
    recommended_song_name = sp.track(recommended_song_id)['name']
    recommended_artist_name = sp.track(recommended_song_id)['artists'][0]['name']

    # Display the recommended song using an embedded player
    print(f"We recommend '{recommended_song_name}' by {recommended_artist_name} based on your selection of '{sp.track(track_id)['name']}'")
    display(play(recommended_song_id))

    # Ask the user if they want more recommendations
    while True:
        choice = input("Would you like more recommendations? (y/n) ")
        if choice == "y":
            recommend(track_id)
            break
        elif choice == "n":
            break
        else:
            print("Invalid input. Please enter 'y' or 'n'.")


## running the code

In [11]:
recommend(search())

Enter the name of a song: sheen rock
1. Sheena Is a Punk Rocker - 2017 Remaster by Ramones
2. Sheena Is a Punk Rocker - 2001 Remaster by Ramones
3. 長く短い祭 by Sheena Ringo
4. Sheena Is a Punk Rocker - 2002 Remaster by Ramones
5. Rain (with Skillibeng) by Shenseea
6. Sheena Is a Punk Rocker by Ramones
7. 宗教 by Sheena Ringo
8. Sheena Is a Punk Rocker - 2009 Remaster by Ramones
9. Shen Ex Anthem by Shenseea
10. Sheena Is a Punk Rocker - 2017 Remaster by Ramones
Please select the song you meant by number (or type 'exit' to cancel): 1
Now playing: Sheena Is a Punk Rocker - 2017 Remaster by Ramones


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


We recommend 'The Gift' by Mark Hollis based on your selection of 'Sheena Is a Punk Rocker - 2017 Remaster'


Would you like more recommendations? (y/n) y


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


...

We recommend 'Pink Moon' by Nick Drake based on your selection of 'Sheena Is a Punk Rocker - 2017 Remaster'


Would you like more recommendations? (y/n) y


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


We recommend 'Frustrated' by R.LUM.R based on your selection of 'Sheena Is a Punk Rocker - 2017 Remaster'


Would you like more recommendations? (y/n) n
